In [1]:
# !sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.3.3/spark-3.3.3-bin-hadoop3.tgz
!tar xf spark-3.3.3-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyspark

# spark-3.2.0-bin-hadoop3.2.tgz  # http://archive.apache.org/dist/spark/

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=9ce9f2cc28710f07d5c6dfea38d5df032e7510e8d7c440631aea48238bc3e71d
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.3-bin-hadoop3"

In [3]:
import findspark
#from pyspark.sql import SparkSession
findspark.init()
from pyspark import SparkContext
import numpy as np
from google.colab import files
files.upload()

Saving sample_1.txt to sample_1.txt


{'sample_1.txt': b'user_id,business_id\r\n---1lKK3aKOuomHnwAkAow,uhb7nSskHRm5RFbGkGlGwQ\r\n---1lKK3aKOuomHnwAkAow,ebVPLAeD7FZ2VzY5ryOCxw\r\n---1lKK3aKOuomHnwAkAow,-kG0N8sBhBotMbu0KVSPaw\r\n---1lKK3aKOuomHnwAkAow,_ucDskZqK5w1QHkoA_nlRw\r\n---1lKK3aKOuomHnwAkAow,6c9e9qH_AdBYQnm7dyiDAQ\r\n---1lKK3aKOuomHnwAkAow,JdXWoFcLDxE7OhC63r8zAQ\r\n---1lKK3aKOuomHnwAkAow,vWFhRvHVIJAzIeOX4g_YcA\r\n---1lKK3aKOuomHnwAkAow,1jpYP-JNOEYCMIzP4OlGuQ\r\n---1lKK3aKOuomHnwAkAow,XlZ2Row0pszDilc5WmXiZQ\r\n---1lKK3aKOuomHnwAkAow,7VWtmHsd4HSNDkF5f5xnng\r\n---1lKK3aKOuomHnwAkAow,QjZFYd5hme7EHegpuJngMQ\r\n---1lKK3aKOuomHnwAkAow,rFRNjwr6Z3r4agQG-LtluA\r\n---1lKK3aKOuomHnwAkAow,n_-CwlqV8fzEIUPky8Ibtw\r\n---1lKK3aKOuomHnwAkAow,ZeLSZRm_l0Aw7idB3_fKbA\r\n---1lKK3aKOuomHnwAkAow,I0f12eU-xFn6Kd_4H57jrQ\r\n---1lKK3aKOuomHnwAkAow,ydcNlRMKs93AQpOhVkSrGQ\r\n---1lKK3aKOuomHnwAkAow,pKk7jCFIm96qDdk0laVT2w\r\n---1lKK3aKOuomHnwAkAow,3vxIqhm2jCVjkUWtmDvTsw\r\n---1lKK3aKOuomHnwAkAow,VUtazCTIc0aoOrQprP_s-Q\r\n---1lKK3aKOuomHnwAkAow,2g_a

In [4]:
def create_adjacency_graph(user_edges_rdd):
    rdd_1 = user_edges_rdd.groupByKey().mapValues(set)
    rdd_2 = user_edges_rdd.map(lambda x: (x[1], x[0])).groupByKey().mapValues(set)
    return rdd_1.union(rdd_2).reduceByKey(lambda x, y: x.union(y))

from itertools import combinations

In [5]:
threshold = int(input("Enter the threshold: "))
#threshold = 4 # the filter threshold to generate edges between user nodes. If threshold is 2, two nodes are connected if they both review at least two same businesses
sc = SparkContext.getOrCreate()
rdd = sc.textFile("sample_1.txt")
N = rdd.count()  # N is the number of rows (lines) in the textfile
print("number of rows: ", N)
rdd = rdd.map(lambda line: line.split(","))
print(rdd.take(2))

Enter the threshold: 6
number of rows:  38649
[['user_id', 'business_id'], ['---1lKK3aKOuomHnwAkAow', 'uhb7nSskHRm5RFbGkGlGwQ']]


In [6]:
  # find (userid, {business1, business2, ...}) and save them in users_rdd:
users_rdd = rdd.map(lambda x: (x[0], x[1])).groupByKey().mapValues(set)
users_rdd.persist()
print("users_rdd take 1:", users_rdd.take(3))

users_rdd take 1: [('user_id', {'business_id'}), ('---1lKK3aKOuomHnwAkAow', {'JVK8szNDoy9MNiYSz_MiAA', 'ebVPLAeD7FZ2VzY5ryOCxw', 'IZrGUQHkxOxyDmifE6p7bw', 'LpyndkMvevIQA-JHoReHNw', 'pKk7jCFIm96qDdk0laVT2w', '8rtd2ZoeNMHjCQQnWuZ-bQ', 'iA8Ve2sZKN5Vz3mYKrtCaQ', 'ik0y2ZYiMQBVQsyDDyOzfw', '4GJc_Ls4f2uVptSfFQTL5g', 'r8Jr-7YfkhJL2nxk9lxUNg', 'VrKja5O6w0aJs-sFPMTtVw', 'pomGBqfbxcqPv14c3XH-ZQ', 'UYop9KYT-_mJsc2bcH83jw', 'XKPosZx2j4RHcZ8Ewdwj0A', '44FSoN5Wj2B9vZTTYHlQZQ', 'eCeFBHucdjbZkJlry8EALw', 'frBAaI0FRj1BwnToT1AFZA', '9IpKGHaPy3hhXo9g5Bt7lg', '3OKwP3ylofhDJDtWJQu5GQ', 'VUtazCTIc0aoOrQprP_s-Q', '02Ycww2etUAxorZb5gIM8Q', 'sG3MhGiT3oSyzHdHLtWRyQ', 'VEFtI92ux6Q9PwMMrm0T-g', 'hD03alkIFsIyoN2rszpeOw', 'n_rEQNgsCOLNfMzGqFhn1g', 'bYGn37k0KgleABKMWgjnMw', '20ib4z2Yo2wlfARFMcFwSQ', 'gJhKRTcfOA-Gn1_R-j-F0w', 'cUwNW5NEua_yMcAx-X2sMg', 'jYqOPpSmtKbKzf0Z_g-Oyg', 'TkSIb5m9KXEBq2hViJwuXg', 'S3kcGHznDcyzsxkOhSZeLg', 'w8zw4-3Ns2M4CQxeW9Bszw', 'kPV1DlXSkcXY5G8r2LLmEA', 'bXCWON2Me0o86qvAb-XZPQ', 'xuAY1qnAMrzk

In [7]:
# create users_dict dictionary (key, value) for fast search, since keys in dict are hashed
users_dict = dict(users_rdd.collect())

# get userid information only:
distinct_users = users_rdd.keys().collect()

print("rdd: number of distinct users:", users_rdd.count())
print("dict: number of distinct users:", len(users_dict))

rdd: number of distinct users: 3375
dict: number of distinct users: 3375


In [8]:
# let us create edges between users:
original_user_edges_list = []  # each item in original_user_edges_list is en edge connecting two users
for temp_user in combinations(distinct_users, 2): # for every two userIDs, check if the number of common businesses they reviewed is >= threshold
  if len(users_dict[temp_user[0]].intersection(users_dict[temp_user[1]])) >= threshold:
    edge_sort_key = tuple(sorted([temp_user[0], temp_user[1]]))
    original_user_edges_list.append(edge_sort_key)

In [9]:
 # each item in user_edges_rdd is en edge connecting two users
user_edges_rdd = sc.parallelize(original_user_edges_list).persist()
print("user_edges_rdd take 2", user_edges_rdd.take(4))

user_edges_rdd take 2 [('---1lKK3aKOuomHnwAkAow', '--2vR0DIsmQ6WfcSzKWigw'), ('---1lKK3aKOuomHnwAkAow', '--YhjyV-ce1nFLYxP49C5A'), ('---1lKK3aKOuomHnwAkAow', '-267Yx8RmdP6io2-qI4UcQ'), ('---1lKK3aKOuomHnwAkAow', '-3s52C4zL_DHRK0ULG6qtg')]


In [10]:
# based on edges, we create an adjacency graph:
user_adjacency_rdd = create_adjacency_graph(user_edges_rdd) #each item in user_adjacency_rdd is (userID, a set of neighbors)
print("user_adjacency_rdd take 2", user_adjacency_rdd.take(2))

user_adjacency_rdd take 2 [('---1lKK3aKOuomHnwAkAow', {'048mif2uzPN800T90sAmyw', '-7bM_DeL2Kj2CuYuVDsLNg', '2t__HKqBFV1-2eIW2Nom9g', '-_2h2cJlBOWAYrfplMU-Cg', '4wp4XI9AxKNqJima-xahlg', '-ARdx8hOcEWlMDjzwLYZ_g', '-RApyq06DMBii7ovXtQnfw', '1Vh2juPPQAw-EYFnulb7cQ', '-Anyb0vB5LrW273whytNRw', '1J-CWvcyB8FDmyZ_GEmvpw', '-Fy91nyOFqPv9M_MaZ4W2g', '3XzhO2aJdvVtYhKOCJsPrg', '2oeFc-L1Ow4UC_j_VOS_kQ', '-TMDrC66dvClx5Z7Hdzrfw', '-Y6tXYPYqeVy37-L5p0rMw', '1GbtKqRpDAfv13fUYIBBmA', '52XZbIuZfVyOhg2YOmFDvA', '--YhjyV-ce1nFLYxP49C5A', '-WMhXcdzVV-o3PN06-yzRA', '-267Yx8RmdP6io2-qI4UcQ', '-ShdX4pDKrldKfic9rHhSQ', '0uNxhZAFbkalQImzJ6UDUA', '-KpEgEen1tj-jdjIS7uVOw', '--2vR0DIsmQ6WfcSzKWigw', '-dbWm5L_Ol2hZeLRoQOK7w', '6gaxwVD2d1C0_qIBOkDkJg', '-PHC1ulwHkY4LEmMqmFwPg', '-kRQLEDxrEBVNXImyAt_ug', '0BoO48jZw2kjJjwgwIjbLw', '-3s52C4zL_DHRK0ULG6qtg'}), ('--2vR0DIsmQ6WfcSzKWigw', {'7qGe49__QqCpk8ZYOm7W8w', '3Z70d8MHB6WhlMpIz9YdZA', '-7bM_DeL2Kj2CuYuVDsLNg', '46-qsKjqR4vptCIYrHH1yw', 'LUWbVukgLd7hGvQ3la4pgg', '0C65

In [11]:
user_adjacency_dict = user_adjacency_rdd.collectAsMap()  # collectAsMap: return the key-value pairs in this RDD to the master as a dictionary.
node_degrees_dict = user_adjacency_rdd.map(lambda x: (x[0], len(x[1]))).collectAsMap() # collectAsMap: return the key-value pairs in this RDD to the master as a dictionary.
print("node_degrees_dict", node_degrees_dict)  # each item in this dict is 'userID' : node-degree
# now we have a graph, saved in user_edges_rdd

node_degrees_dict {'---1lKK3aKOuomHnwAkAow': 30, '--2vR0DIsmQ6WfcSzKWigw': 72, '--RlSfc-QmcHFGHyX6aVjA': 6, '--YhjyV-ce1nFLYxP49C5A': 51, '-0HhZbPBlB1YZx3BhAfaEA': 12, '-267Yx8RmdP6io2-qI4UcQ': 46, '-2kCxY7_aw5hOz7fJnGMbQ': 3, '-3s52C4zL_DHRK0ULG6qtg': 8, '-50XWnmQGqBgEI-9ANvLlg': 1, '-6559fkJ6rCWIZDbqVUomA': 1, '-7bM_DeL2Kj2CuYuVDsLNg': 15, '-8syaSLDbMXW2F2FApVgig': 1, '-9b4s874f_CnznTu4JorRg': 27, '-ARdx8hOcEWlMDjzwLYZ_g': 2, '-Anyb0vB5LrW273whytNRw': 41, '-EJorVxe7h2GSxdiRyMmDA': 7, '-Fy91nyOFqPv9M_MaZ4W2g': 7, '-KpEgEen1tj-jdjIS7uVOw': 22, '-OY3fkHVYy0Dx160rKCiWQ': 5, '-P3SyBLmBhyhDcYatlBgBQ': 10, '-PHC1ulwHkY4LEmMqmFwPg': 19, '-R3Liu8xDWu-VpWSW8YlKA': 7, '-RA9NLalwmRTOX_8UMHnVQ': 25, '-RApyq06DMBii7ovXtQnfw': 4, '-ShdX4pDKrldKfic9rHhSQ': 11, '-TMDrC66dvClx5Z7Hdzrfw': 16, '-VNv58eLhbQpz787rcD8VA': 11, '-WMhXcdzVV-o3PN06-yzRA': 14, '-Y6tXYPYqeVy37-L5p0rMw': 9, '-YV1yESQXqR3vpIgBjKDsw': 1, '-ZqPTpx9TyyvdAWbajdilA': 6, '-_2h2cJlBOWAYrfplMU-Cg': 4, '-k5yFUChotBMGQgHdFZvLQ': 12, '-kMn9K

In [12]:
print(user_adjacency_rdd.count())

357


In [13]:

user_adjacency_data = user_adjacency_rdd.collect()

# Create a mapping of user IDs to unique integer indices
userid_convert_index = {}
indexid_convert_userid = {}
for index, (userid, _) in enumerate(user_adjacency_data):
    userid_convert_index[userid] = index
    indexid_convert_userid[index] = userid

# Define the number of communities and other parameters
initial_community = 5
hyperparameter_term = 0.1
max_iter = 100
learning_rate = 0.01

# Define the number of nodes based on the list size
number_of_nodes = len(user_adjacency_data)

# Initialize the F matrix with random values
F = np.random.rand(number_of_nodes, initial_community)



In [14]:
def calculate_gradient(F, adjacency_matrix, hyperparameter_term):
    number_of_nodes, initial_community = F.shape
    likelihood = 0
    gradient = np.zeros((number_of_nodes, initial_community))

    for u in range(number_of_nodes):
        F_u = F[u]  # Community membership vector for node u
        nei_indices = [userid_convert_index[neighbor] for neighbor in user_adjacency_data[u][1] if neighbor in userid_convert_index]

        exponential_sum = np.zeros((initial_community,))
        sum_not_in_neighbors = np.zeros((initial_community,))

        for v in nei_indices:
            F_v = F[v]
            exponential = np.exp(-np.dot(F_u, F_v))
            exponential = np.clip(exponential, 1e-15, 1e15)  # Clip values to prevent overflow
            exponential_sum += F_v * exponential / (1 - exponential)

        likelihood -= np.dot(exponential_sum, F_u)

        # Compute the gradient
        gradient[u] = exponential_sum / np.linalg.norm(exponential_sum)

    # Add regularization term
    gradient -= hyperparameter_term * F

    return likelihood, gradient

In [15]:
adjacency_matrix = np.zeros((number_of_nodes, number_of_nodes))
for u, (userid, neighbors) in enumerate(user_adjacency_data):
    for neighbor in neighbors:
        if neighbor in userid_convert_index:
            v = userid_convert_index[neighbor]
            adjacency_matrix[u, v] = 1

# Optimization
for iteration in range(max_iter):
    likelihood, gradient = calculate_gradient(F, adjacency_matrix, hyperparameter_term)

    for u in range(number_of_nodes):
        # Calculate the gradient for row u while keeping others fixed
        F_u = F[u]  # Community membership vector for node u
        gradient_u = gradient[u]

        # Update Fu using gradient ascent
        F_u += learning_rate * gradient_u

        # Project Fu to be non-negative
        F_u[F_u < 0] = 0


In [16]:
print(F)


[[0.76961504 1.11896538 0.88143138 1.27928725 1.305825  ]
 [0.44081337 0.65317563 1.26278904 0.44444989 1.25417546]
 [0.84901015 1.04259247 1.37662841 1.12944442 1.17627837]
 ...
 [0.92045425 0.2497108  0.41960555 0.53191263 1.28622277]
 [0.76286624 1.31086904 0.70755859 0.94901491 1.59265039]
 [1.14339417 0.95202945 1.00484104 1.13942016 0.18641062]]


In [17]:
def nodes_community(Fu):
    community = np.argmax(Fu)
    return community

In [18]:
import math


In [19]:
# Calculate the threshold δ based on ε
e = 0.1  # Replace with your desired value of ε
threshold_delta = math.sqrt(-math.log(1 - e))

# Determine the community membership based on the threshold δ
community_dict = {}  # Dictionary to store the counts of nodes in each community

for node in range(F.shape[0]):
    Fu = F[node, :]
    community = nodes_community(Fu)
    if Fu[community] >= threshold_delta:
        print(f"User {node} belongs to Community {community}")
        # Update the community count
        if community in community_dict:
            community_dict[community] += 1
        else:
            community_dict[community] = 1
    else:
        print(f"User {node} does not belong to any community")

# Print the number of nodes in each community
print("Number of Nodes in Each Community:")
for community, count in community_dict.items():
    print(f"Community {community}: {count} nodes")

User 0 belongs to Community 4
User 1 belongs to Community 2
User 2 belongs to Community 2
User 3 belongs to Community 0
User 4 belongs to Community 1
User 5 belongs to Community 2
User 6 belongs to Community 1
User 7 belongs to Community 4
User 8 belongs to Community 1
User 9 belongs to Community 2
User 10 belongs to Community 1
User 11 belongs to Community 1
User 12 belongs to Community 3
User 13 belongs to Community 3
User 14 belongs to Community 0
User 15 belongs to Community 4
User 16 belongs to Community 4
User 17 belongs to Community 2
User 18 belongs to Community 2
User 19 belongs to Community 3
User 20 belongs to Community 3
User 21 belongs to Community 4
User 22 belongs to Community 2
User 23 belongs to Community 3
User 24 belongs to Community 4
User 25 belongs to Community 1
User 26 belongs to Community 0
User 27 belongs to Community 2
User 28 belongs to Community 2
User 29 belongs to Community 0
User 30 belongs to Community 4
User 31 belongs to Community 0
User 32 belongs to

In [20]:
def modularity_calculation(F, adjacency_matrix, community_assignments):
    number_of_nodes, initial_community = F.shape
    modularity = 0

    for u in range(number_of_nodes):
        for v in range(number_of_nodes):
            A_uv = adjacency_matrix[u, v]
            k_u = np.sum(adjacency_matrix[u, :])
            k_v = np.sum(adjacency_matrix[v, :])
            delta_c_uv = int(community_assignments[u] == community_assignments[v])

            modularity += (A_uv - (k_u * k_v) / (2 * M)) * delta_c_uv

    modularity /= (2 * M)

    return modularity

# Variables for modularity calculation
M = np.sum(adjacency_matrix)  # Total number of edges
best_modularity = -1  # Initialize with a low value
best_initial_community = -1

# Iterate over different community counts
for initial_community in range(2, 7):  # You can adjust the range as needed
    # Initialize the F matrix with random values
    F = np.random.rand(number_of_nodes, initial_community)


    # Optimization
    for iteration in range(max_iter):
        likelihood, gradient = calculate_gradient(F, adjacency_matrix, hyperparameter_term)

        for u in range(number_of_nodes):
        # Calculate the gradient for row u while keeping others fixed
            F_u = F[u]  # Community membership vector for node u
            gradient_u = gradient[u]

        # Update Fu using gradient ascent
            F_u += learning_rate * gradient_u

        # Project Fu to be non-negative
            F_u[F_u < 0] = 0

    # Extract community assignments from F matrix
    community_assignments = np.argmax(F, axis=1)
  # Calculate modularity for the current community count
    modularity = modularity_calculation(F, adjacency_matrix, community_assignments)

    # Check if this is the best modularity so far
    if modularity > best_modularity:
        best_modularity = modularity
        best_initial_community = initial_community

# Print the best number of communities and the corresponding modularity
print("Best number of communities:", best_initial_community)
print("Best modularity:", best_modularity)


Best number of communities: 2
Best modularity: 0.15533599305325924
